In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
!pip install transformers==4.33.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.50.0
    Uninstalling transformers-4.50.0:
      Successfully uninstalled transformers-4.50.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.33.1 which is incompatible.


In [6]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=1d129866151feca7701db1fb63782dbe86a47f7da37127d2511b4baa78b99f6d
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [7]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [ ]:
!pip install huggingface_hub

In [ ]:
!pip show transformers

Name: transformers
Version: 4.38.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers


In [ ]:
# !GIT_SSH_COMMAND="ssh -i /path/to/your/ssh_key" git clone git@hf.co:username/model-repo.git && cd model-repo && git lfs pull

In [8]:
!git lfs install

Git LFS initialized.


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!git clone https://huggingface.co/facebook/layerskip-llama2-7B

In [9]:
import torch
from draftandverify.modeling_llama import LlamaForCausalLM
from transformers import AutoTokenizer
from datasets import load_dataset

SEED = 1274

/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


(Re-)Loading modeling...


In [ ]:
torch.nn.Linear.reset_parameters = lambda x: None
model = LlamaForCausalLM.from_pretrained('meta-llama/Llama-2-7b-hf', torch_dtype=torch.bfloat16, token="")
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf', token="")

# model = LlamaForCausalLM.from_pretrained('facebook/layerskip-llama2-7B', torch_dtype=torch.bfloat16, token="")
# tokenizer = AutoTokenizer.from_pretrained('facebook/layerskip-llama2-7B', token="")

In [51]:
model = model.to('cuda:0').eval()

In [15]:
cnn = load_dataset('cnn_dailymail', '3.0.0').shuffle(SEED)
prompts = []

for i in range(4):
    item = cnn['train'][i+100]
    cnn_context = 'Article: ' + item['article'] + '\nSummary: ' + item['highlights'].replace('\n', '')

    item = cnn['train'][i]
    prompt = cnn_context + '\nArticle: ' + item['article'] + '\nSummary:'
    prompts.append(prompt)

In [16]:
!pip install bayesian-optimization

In [49]:
# Search implementation borrowed from: 
# https://github.com/dilab-zju/self-speculative-decoding/tree/main

from draftandverify.searching import LayerSkippingSearching

In [54]:
layer_searching = LayerSkippingSearching(model, tokenizer, prompts, evaluate_config={"generate_fn": "essg", "max_new_tokens": 32})

In [55]:
layer_searching.probe([8,10,15,18,20,24,25,26,27,28,29,30,31], [])
layer_searching.probe([3, 5, 6, 8, 10, 11, 14, 15, 18, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31], [6, 9, 10, 11, 15, 24, 25, 27, 28])

In [ ]:
layer_searching.search(100)

|   iter    |  target   |    x0     |    x1     |    x10    |    x11    |    x12    |    x13    |    x14    |    x15    |    x16    |    x17    |    x18    |    x19    |    x2     |    x20    |    x21    |    x22    |    x23    |    x24    |    x25    |    x26    |    x27    |    x28    |    x29    |    x3     |    x30    |    x31    |    x32    |    x33    |    x34    |    x35    |    x36    |    x37    |    x38    |    x39    |    x4     |    x40    |    x41    |    x42    |    x43    |    x44    |    x45    |    x46    |    x47    |    x48    |    x49    |    x5     |    x50    |    x51    |    x52    |    x53    |    x54    |    x55    |    x56    |    x57    |    x58    |    x59    |    x6     |    x60    |    x61    |    x62    |    x63    |    x7     |    x8     |    x9     |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [56]:
# regular llama
layer_searching.search(100)

|   iter    |  target   |    x0     |    x1     |    x10    |    x11    |    x12    |    x13    |    x14    |    x15    |    x16    |    x17    |    x18    |    x19    |    x2     |    x20    |    x21    |    x22    |    x23    |    x24    |    x25    |    x26    |    x27    |    x28    |    x29    |    x3     |    x30    |    x31    |    x32    |    x33    |    x34    |    x35    |    x36    |    x37    |    x38    |    x39    |    x4     |    x40    |    x41    |    x42    |    x43    |    x44    |    x45    |    x46    |    x47    |    x48    |    x49    |    x5     |    x50    |    x51    |    x52    |    x53    |    x54    |    x55    |    x56    |    x57    |    x58    |    x59    |    x6     |    x60    |    x61    |    x62    |    x63    |    x7     |    x8     |    x9     |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

([2, 5, 8, 9, 14, 15, 18, 20, 22, 23, 24, 25, 27, 28, 29, 30],
 [7, 8, 9, 14, 19, 23, 29])

In [42]:
layer_searching.get_solution()

([5, 6, 8, 10, 15, 18, 19, 20, 24, 25, 26, 27, 28, 29, 30],
 [0, 8, 11, 21, 22, 27, 28, 31])

In [46]:
# layerskip llama 2
layer_searching.get_solution()

([2, 4, 9, 10, 12, 15, 18, 19, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31],
 [2, 8, 9, 13, 15, 16, 22, 23, 27, 29, 30, 31])

In [57]:
# regular llama2
layer_searching.get_solution()

([2, 5, 8, 9, 14, 15, 18, 20, 22, 23, 24, 25, 27, 28, 29, 30],
 [7, 8, 9, 14, 19, 23, 29])